# PATH SETUP 

In [19]:
from pathlib import Path
import sys

try:
    # Running as .py script
    FILE_PATH = Path(__file__).resolve()
    PROJECT_ROOT = FILE_PATH.parents[2]  # crypto-volatility-ml
except NameError:
    # Running inside Jupyter (cwd = crypto-volatility-ml/notebooks)
    PROJECT_ROOT = Path.cwd().resolve().parents[0]

DATA_DIR = PROJECT_ROOT / "data"
PROCESSED_DIR = DATA_DIR / "processed"
REPORT_DIR = PROJECT_ROOT / "reports"
FIG_DIR = REPORT_DIR / "figures"

# Create folders if missing
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("PROCESSED_DIR:", PROCESSED_DIR)
print("FIG_DIR:", FIG_DIR)


PROJECT_ROOT: C:\Users\BALA\OneDrive - University of Hertfordshire\Desktop\Hemanth Project\crypto-volatility-ml
PROCESSED_DIR: C:\Users\BALA\OneDrive - University of Hertfordshire\Desktop\Hemanth Project\crypto-volatility-ml\data\processed
FIG_DIR: C:\Users\BALA\OneDrive - University of Hertfordshire\Desktop\Hemanth Project\crypto-volatility-ml\reports\figures


## # AUTO ARIMA MODEL — FULL PIPELINE WITH DETAILED LOGGING (p,d,q)

In [2]:
# ============================================================
# IMPROVED AUTO-ARIMA WITH LAG REGRESSORS (NO LEAKAGE)
# ============================================================

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from pmdarima import auto_arima

# ---------- Paths ----------
PROJECT_ROOT = Path("..").resolve()   # adjust if needed
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
FIG_DIR = PROJECT_ROOT / "reports" / "figures"

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

# ---------- Load full engineered data ----------
df = pd.read_csv(PROCESSED_DIR / "crypto_features.csv", parse_dates=["Date"])
df = df.sort_values(["Name", "Date"]).reset_index(drop=True)

print("Dataset shape:", df.shape)
print("Columns:", df.columns.tolist())

# ---------- Helper: plotting & saving ----------
def save_forecast_plot(dates, actual, predicted, coin, model_name):
    plt.figure(figsize=(11, 5))
    plt.plot(dates, actual, label="Actual", linewidth=2)
    plt.plot(dates, predicted, label="Predicted", linewidth=2)
    plt.title(f"{model_name} Forecast vs Actual — {coin}")
    plt.xlabel("Date")
    plt.ylabel("Close Price")
    plt.legend()
    plt.tight_layout()
    out_path = FIG_DIR / f"{model_name}_{coin}_forecast.png"
    plt.savefig(out_path, dpi=240)
    plt.close()
    return out_path

def save_residual_plot(dates, residuals, coin, model_name):
    plt.figure(figsize=(11, 5))
    plt.plot(dates, residuals, color="purple")
    plt.axhline(0, color="black", linestyle="--")
    plt.title(f"{model_name} Residuals — {coin}")
    plt.xlabel("Date")
    plt.ylabel("Residual (y_true - y_pred)")
    plt.tight_layout()
    out_path = FIG_DIR / f"{model_name}_{coin}_residuals.png"
    plt.savefig(out_path, dpi=240)
    plt.close()
    return out_path

def save_predictions_csv(dates, actual, predicted, coin, model_name):
    out = pd.DataFrame({
        "Date": dates,
        "y_true": actual,
        "y_pred": predicted
    })
    out_path = PROCESSED_DIR / f"predictions_{model_name.lower()}_{coin}.csv"
    out.to_csv(out_path, index=False)
    return out_path

# ---------- ARIMA runner ----------
def run_auto_arima_exog(sub_df, coin_name, exog_cols):
    """
    sub_df: one-coin dataframe
    exog_cols: list of lag/time features safe as regressors
    """
    # Keep only required columns
    cols = ["Date", "Close"] + exog_cols
    sub = sub_df[cols].dropna().copy()

    if len(sub) < 300:
        print(f" Skipping {coin_name}: not enough rows after dropping NaNs.")
        return None

    # Time-based split 80/20
    split = int(len(sub) * 0.8)
    train = sub.iloc[:split]
    test  = sub.iloc[split:]

    print(f"• Train interval: {train['Date'].iloc[0].date()} → {train['Date'].iloc[-1].date()} ({len(train)} rows)")
    print(f"• Test interval : {test['Date'].iloc[0].date()} → {test['Date'].iloc[-1].date()} ({len(test)} rows)")

    y_train = train["Close"].values
    y_test  = test["Close"].values

    X_train = train[exog_cols].values
    X_test  = test[exog_cols].values

    # ---------- Fit Auto ARIMA ----------
    print(f" Fitting Auto ARIMA with exogenous regressors for {coin_name}...")

    model = auto_arima(
        y_train,
        exogenous=X_train,
        seasonal=False,
        stepwise=True,
        max_p=3,
        max_q=3,
        max_d=2,
        max_order=6,
        suppress_warnings=True,
        error_action="ignore",
        trace=False,
        n_jobs=1
    )

    order = model.order  # (p,d,q)
    p, d, q = order
    print(f" Selected ARIMA order (p,d,q) = ({p}, {d}, {q})")

    # ---------- Train (in-sample) metrics ----------
    train_pred = model.predict_in_sample(exogenous=X_train)

    train_mae = mean_absolute_error(y_train, train_pred)
    train_rmse = root_mean_squared_error(y_train, train_pred)
    print(f" Train MAE  = {train_mae:.3f}")
    print(f" Train RMSE = {train_rmse:.3f}")

    # ---------- Test forecast ----------
    print(f" Forecasting next {len(test)} points...")
    test_pred = model.predict(n_periods=len(test), exogenous=X_test)

    test_mae = mean_absolute_error(y_test, test_pred)
    test_rmse = root_mean_squared_error(y_test, test_pred)
    print(f" Test MAE   = {test_mae:.3f}")
    print(f" Test RMSE  = {test_rmse:.3f}")

    # ---------- Save plots & CSV ----------
    forecast_plot = save_forecast_plot(
        dates=test["Date"].values,
        actual=y_test,
        predicted=test_pred,
        coin=coin_name,
        model_name="AutoARIMA_Exog"
    )

    residuals = y_test - test_pred
    residual_plot = save_residual_plot(
        dates=test["Date"].values,
        residuals=residuals,
        coin=coin_name,
        model_name="AutoARIMA_Exog"
    )

    pred_csv = save_predictions_csv(
        dates=test["Date"].values,
        actual=y_test,
        predicted=test_pred,
        coin=coin_name,
        model_name="AutoARIMA_Exog"
    )

    return {
        "Coin": coin_name,
        "Model": "AutoARIMA_Exog",
        "p": p,
        "d": d,
        "q": q,
        "Train_MAE": train_mae,
        "Train_RMSE": train_rmse,
        "Test_MAE": test_mae,
        "Test_RMSE": test_rmse,
        "ForecastPlot": str(forecast_plot),
        "ResidualPlot": str(residual_plot),
        "PredCSV": str(pred_csv),
    }


# ============================================================
# RUN AUTO ARIMA WITH EXOGENOUS REGRESSORS
# ============================================================

# Safe regressors: ONLY lags + calendar features (no leakage)
arima_exog_cols = [
    "Close_lag1", "Close_lag7", "Close_lag14", "Close_lag30",
    "Return_lag1", "Return_lag7", "Return_lag14", "Return_lag30",
    "Volume_lag1", "Volume_lag7", "Volume_lag14", "Volume_lag30",
    "DayOfWeek", "Month"
]

results_arima_exog = []

print("\n================= AUTO ARIMA (WITH EXOGENOUS) START =================\n")

for coin in df["Name"].unique():
    print("----------------------------------------------------------")
    print(f" Now Processing Coin: {coin.upper()}")
    print("----------------------------------------------------------")

    sub_coin = df[df["Name"] == coin].copy()

    try:
        row = run_auto_arima_exog(sub_coin, coin, arima_exog_cols)
        if row is not None:
            results_arima_exog.append(row)
    except Exception as e:
        print(f" !! Failed for {coin}: {e}")

print("\n================= AUTO ARIMA (WITH EXOGENOUS) FINISHED =================\n")

if results_arima_exog:
    arima_exog_df = pd.DataFrame(results_arima_exog).sort_values("Test_RMSE")
    display(arima_exog_df)
else:
    print("No ARIMA results — something went wrong.")


Dataset shape: (37082, 44)
Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Name', 'Symbol', 'SourceFile', 'LogReturn', 'Return_%', 'Volatility_7d', 'Volatility_30d', 'Momentum_7d', 'Momentum_30d', 'RSI_14', 'EMA_12', 'EMA_26', 'MACD', 'MACD_Signal', 'EMA_10', 'EMA_20', 'EMA_50', 'BB_Upper', 'BB_Lower', 'BB_Width', 'High_Low_%', 'Close_Open_%', 'MarketPressure', 'Close_lag1', 'Volume_lag1', 'Return_lag1', 'Close_lag7', 'Volume_lag7', 'Return_lag7', 'Close_lag14', 'Volume_lag14', 'Return_lag14', 'Close_lag30', 'Volume_lag30', 'Return_lag30', 'DayOfWeek', 'Month', 'Quarter']

================= AUTO ARIMA (WITH EXOGENOUS) START =================

----------------------------------------------------------
 Now Processing Coin: AAVE
----------------------------------------------------------
 Skipping Aave: not enough rows after dropping NaNs.
----------------------------------------------------------
 Now Processing Coin: BINANCE COIN
--------------------------------------------

,Coin,Model,p,d,q,Train_MAE,Train_RMSE,Test_MAE,Test_RMSE,ForecastPlot,ResidualPlot,PredCSV
17,USD Coin,AutoARIMA_Exog,0,1,2,0.003937,0.036455,0.000456,0.001016,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
16,Tether,AutoARIMA_Exog,2,1,1,0.002887,0.023950,0.001694,0.002454,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
15,TRON,AutoARIMA_Exog,3,1,3,0.001645,0.004624,0.033432,0.049419,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
5,Crypto.com Coin,AutoARIMA_Exog,0,1,0,0.002320,0.004106,0.067442,0.083893,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
6,Dogecoin,AutoARIMA_Exog,1,1,0,0.000078,0.000220,0.060954,0.142855,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
12,NEM,AutoARIMA_Exog,2,1,3,0.007073,0.027819,0.151176,0.214900,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
14,Stellar,AutoARIMA_Exog,3,1,3,0.004107,0.013509,0.141974,0.221459,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
19,XRP,AutoARIMA_Exog,1,1,1,0.009734,0.040190,0.217128,0.402317,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
9,IOTA,AutoARIMA_Exog,2,1,2,0.043218,0.108275,0.623414,0.892742,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
2,Cardano,AutoARIMA_Exog,0,1,3,0.007048,0.019612,0.722350,0.941808,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...


## PROPHET MODEL

In [3]:
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from prophet import Prophet

# ---------- Paths ----------
PROJECT_ROOT = Path("..").resolve()
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
FIG_DIR = PROJECT_ROOT / "reports" / "figures"

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

# ---------- Load engineered dataset ----------
df = pd.read_csv(PROCESSED_DIR / "crypto_features.csv", parse_dates=["Date"])
df = df.sort_values(["Name", "Date"]).reset_index(drop=True)

print("Dataset shape:", df.shape)
print("Columns:", df.columns.tolist())

# ---------- Helper: plotting & saving ----------
def save_forecast_plot(dates, actual, predicted, coin, model_name):
    plt.figure(figsize=(11, 5))
    plt.plot(dates, actual, label="Actual", linewidth=2)
    plt.plot(dates, predicted, label="Predicted", linewidth=2)
    plt.title(f"{model_name} Forecast vs Actual — {coin}")
    plt.xlabel("Date")
    plt.ylabel("Close Price")
    plt.legend()
    plt.tight_layout()
    out_path = FIG_DIR / f"{model_name}_{coin}_forecast.png"
    plt.savefig(out_path, dpi=240)
    plt.close()
    return out_path

def save_residual_plot(dates, residuals, coin, model_name):
    plt.figure(figsize=(11, 5))
    plt.plot(dates, residuals, color="purple")
    plt.axhline(0, color="black", linestyle="--")
    plt.title(f"{model_name} Residuals — {coin}")
    plt.xlabel("Date")
    plt.ylabel("Residual (y_true - y_pred)")
    plt.tight_layout()
    out_path = FIG_DIR / f"{model_name}_{coin}_residuals.png"
    plt.savefig(out_path, dpi=240)
    plt.close()
    return out_path

def save_predictions_csv(dates, actual, predicted, coin, model_name):
    out = pd.DataFrame({
        "Date": dates,
        "y_true": actual,
        "y_pred": predicted
    })
    out_path = PROCESSED_DIR / f"predictions_{model_name.lower()}_{coin}.csv"
    out.to_csv(out_path, index=False)
    return out_path


# ---------- Prophet runner ----------
def run_prophet_with_lags(sub_df, coin_name, regressors):
    """
    sub_df: one-coin dataframe
    regressors: lag + calendar features only (no leakage)
    """
    cols = ["Date", "Close"] + regressors
    sub = sub_df[cols].dropna().copy()

    if len(sub) < 300:
        print(f" Skipping {coin_name}: not enough rows after dropping NaNs.")
        return None

    # Time-based 80/20 split
    split = int(len(sub) * 0.8)
    train = sub.iloc[:split]
    test  = sub.iloc[split:]

    print(f"Train interval: {train['Date'].iloc[0].date()} → {train['Date'].iloc[-1].date()} ({len(train)} rows)")
    print(f"Test interval : {test['Date'].iloc[0].date()} → {test['Date'].iloc[-1].date()} ({len(test)} rows)")

    # Prepare Prophet dfs
    def to_prophet(df_part):
        tmp = df_part.rename(columns={"Date": "ds", "Close": "y"})
        return tmp

    train_p = to_prophet(train)
    test_p  = to_prophet(test)

    # Simple hyperparameter grid
    cp_grid = [0.3, 0.8]      # changepoint_prior_scale
    sp_grid = [5.0, 10.0]     # seasonality_prior_scale

    best_cfg = None
    best_rmse = np.inf

    print(" Tuning hyperparameters on train set (using last 20% as validation)...")

    # Split train into base_train + validation
    n_train = len(train_p)
    n_val = max(int(0.2 * n_train), 60)
    base_train = train_p.iloc[:-n_val]
    val = train_p.iloc[-n_val:]

    base_train_X = base_train[["ds"] + regressors]
    val_X = val[["ds"] + regressors]

    for cp in cp_grid:
        for sp in sp_grid:
            m = Prophet(
                seasonality_mode="multiplicative",
                changepoint_prior_scale=cp,
                seasonality_prior_scale=sp,
                changepoint_range=0.95,
                weekly_seasonality=True,
                yearly_seasonality=False,
                daily_seasonality=True,
            )

            for r in regressors:
                m.add_regressor(r)

            m.fit(base_train)

            val_future = val_X.copy()
            val_forecast = m.predict(val_future)
            y_val_pred = val_forecast["yhat"].values
            y_val_true = val["y"].values

            val_rmse = root_mean_squared_error(y_val_true, y_val_pred)
            print(f"   cp={cp}, sp={sp} → Val RMSE = {val_rmse:.2f}")

            if val_rmse < best_rmse:
                best_rmse = val_rmse
                best_cfg = (cp, sp)

    cp_best, sp_best = best_cfg
    print(f" Best config for {coin_name}: cp={cp_best}, sp={sp_best} (Val RMSE={best_rmse:.2f})")

    # ---------- model on full train ----------
    m_final = Prophet(
        seasonality_mode="multiplicative",
        changepoint_prior_scale=cp_best,
        seasonality_prior_scale=sp_best,
        changepoint_range=0.95,
        weekly_seasonality=True,
        yearly_seasonality=False,
        daily_seasonality=True,
    )

    for r in regressors:
        m_final.add_regressor(r)

    m_final.fit(train_p)

    # Train metrics
    train_future = train_p[["ds"] + regressors]
    train_forecast = m_final.predict(train_future)
    y_train_pred = train_forecast["yhat"].values
    y_train_true = train_p["y"].values

    train_mae = mean_absolute_error(y_train_true, y_train_pred)
    train_rmse = root_mean_squared_error(y_train_true, y_train_pred)
    print(f"  Train MAE  = {train_mae:.3f}")
    print(f"  Train RMSE = {train_rmse:.3f}")

    # Test forecast
    test_future = test_p[["ds"] + regressors]
    test_forecast = m_final.predict(test_future)
    y_test_pred = test_forecast["yhat"].values
    y_test_true = test_p["y"].values

    test_mae = mean_absolute_error(y_test_true, y_test_pred)
    test_rmse = root_mean_squared_error(y_test_true, y_test_pred)
    print(f" Test MAE   = {test_mae:.3f}")
    print(f" Test RMSE  = {test_rmse:.3f}")

    # Save plots & CSV
    forecast_plot = save_forecast_plot(
        dates=test["Date"].values,
        actual=y_test_true,
        predicted=y_test_pred,
        coin=coin_name,
        model_name="Prophet_LagReg"
    )

    residuals = y_test_true - y_test_pred
    residual_plot = save_residual_plot(
        dates=test["Date"].values,
        residuals=residuals,
        coin=coin_name,
        model_name="Prophet_LagReg"
    )

    pred_csv = save_predictions_csv(
        dates=test["Date"].values,
        actual=y_test_true,
        predicted=y_test_pred,
        coin=coin_name,
        model_name="Prophet_LagReg"
    )

    return {
        "Coin": coin_name,
        "Model": "Prophet_LagReg",
        "cp_best": cp_best,
        "sp_best": sp_best,
        "Val_RMSE": best_rmse,
        "Train_MAE": train_mae,
        "Train_RMSE": train_rmse,
        "Test_MAE": test_mae,
        "Test_RMSE": test_rmse,
        "ForecastPlot": str(forecast_plot),
        "ResidualPlot": str(residual_plot),
        "PredCSV": str(pred_csv),
    }


# ============================================================
# RUN PROPHET WITH LAG REGRESSORS
# ============================================================

prophet_regressors = [
    "Close_lag1", "Close_lag7", "Close_lag14", "Close_lag30",
    "Return_lag1", "Return_lag7", "Return_lag14", "Return_lag30",
    "Volume_lag1", "Volume_lag7", "Volume_lag14", "Volume_lag30",
    "DayOfWeek", "Month"
]

results_prophet_lag = []

print("\n================= PROPHET (LAG REGRESSORS) START =================\n")

for coin in df["Name"].unique():
    print("----------------------------------------------------------")
    print(f" Now Processing Coin: {coin.upper()}")
    print("----------------------------------------------------------")

    sub_coin = df[df["Name"] == coin].copy()

    try:
        row = run_prophet_with_lags(sub_coin, coin, prophet_regressors)
        if row is not None:
            results_prophet_lag.append(row)
    except Exception as e:
        print(f" !! Failed for {coin}: {e}")

print("\n================= PROPHET (LAG REGRESSORS) FINISHED =================\n")

if results_prophet_lag:
    prophet_lag_df = pd.DataFrame(results_prophet_lag).sort_values("Test_RMSE")
    display(prophet_lag_df)
else:
    print("No Prophet results — something went wrong.")


Dataset shape: (37082, 44)
Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Name', 'Symbol', 'SourceFile', 'LogReturn', 'Return_%', 'Volatility_7d', 'Volatility_30d', 'Momentum_7d', 'Momentum_30d', 'RSI_14', 'EMA_12', 'EMA_26', 'MACD', 'MACD_Signal', 'EMA_10', 'EMA_20', 'EMA_50', 'BB_Upper', 'BB_Lower', 'BB_Width', 'High_Low_%', 'Close_Open_%', 'MarketPressure', 'Close_lag1', 'Volume_lag1', 'Return_lag1', 'Close_lag7', 'Volume_lag7', 'Return_lag7', 'Close_lag14', 'Volume_lag14', 'Return_lag14', 'Close_lag30', 'Volume_lag30', 'Return_lag30', 'DayOfWeek', 'Month', 'Quarter']

================= PROPHET (LAG REGRESSORS) START =================

----------------------------------------------------------
 Now Processing Coin: AAVE
----------------------------------------------------------
 Skipping Aave: not enough rows after dropping NaNs.
----------------------------------------------------------
 Now Processing Coin: BINANCE COIN
-----------------------------------------------

04:29:08 - cmdstanpy - INFO - Chain [1] start processing
04:29:16 - cmdstanpy - INFO - Chain [1] done processing


   cp=0.3, sp=5.0 → Val RMSE = 8.58


04:29:17 - cmdstanpy - INFO - Chain [1] start processing
04:29:21 - cmdstanpy - INFO - Chain [1] done processing


   cp=0.3, sp=10.0 → Val RMSE = 7.33


...

04:33:03 - cmdstanpy - INFO - Chain [1] start processing
04:33:05 - cmdstanpy - INFO - Chain [1] done processing


   cp=0.8, sp=10.0 → Val RMSE = 0.10
 Best config for XRP: cp=0.3, sp=10.0 (Val RMSE=0.09)


04:33:06 - cmdstanpy - INFO - Chain [1] start processing
04:33:08 - cmdstanpy - INFO - Chain [1] done processing


  Train MAE  = 0.013
  Train RMSE = 0.038
 Test MAE   = 0.277
 Test RMSE  = 0.458

================= PROPHET (LAG REGRESSORS) FINISHED =================



,Coin,Model,cp_best,sp_best,Val_RMSE,Train_MAE,Train_RMSE,Test_MAE,Test_RMSE,ForecastPlot,ResidualPlot,PredCSV
16,Tether,Prophet_LagReg,0.3,5.0,0.006735,0.002443,0.005062,0.002717,0.003787,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
17,USD Coin,Prophet_LagReg,0.8,5.0,0.010558,0.002566,0.004107,0.010536,0.011466,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
5,Crypto.com Coin,Prophet_LagReg,0.3,10.0,0.006156,0.002334,0.003770,0.012213,0.016095,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
15,TRON,Prophet_LagReg,0.3,10.0,0.003433,0.001895,0.004636,0.014711,0.019107,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
6,Dogecoin,Prophet_LagReg,0.3,10.0,0.000112,0.000084,0.000216,0.007662,0.023727,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
14,Stellar,Prophet_LagReg,0.3,5.0,0.019263,0.004725,0.013352,0.067640,0.101605,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
9,IOTA,Prophet_LagReg,0.8,10.0,0.046258,0.047076,0.103736,0.110564,0.155057,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
12,NEM,Prophet_LagReg,0.3,5.0,0.012285,0.009407,0.026007,0.126985,0.175865,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
2,Cardano,Prophet_LagReg,0.3,5.0,0.033909,0.007698,0.017939,0.260808,0.365844,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
19,XRP,Prophet_LagReg,0.3,10.0,0.090628,0.012952,0.037814,0.277163,0.457708,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...


In [4]:
# ============================================================
# IMPROVED LSTM FORECASTING — MULTIVARIATE, REGULARISED
# ============================================================

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# ----------------- Paths -----------------
PROJECT_ROOT = Path("..").resolve()
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
FIG_DIR = PROJECT_ROOT / "reports" / "figures"

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

# ----------------- Load data -----------------
df = pd.read_csv(PROCESSED_DIR / "crypto_features.csv", parse_dates=["Date"])
df = df.sort_values(["Name", "Date"]).reset_index(drop=True)

print("Dataset shape:", df.shape)
print("Columns:", df.columns.tolist())

# ----------------- Config -----------------
TARGET_COL = "Close"
LOOKBACK = 60       # days in the input sequence
HORIZON = 1         # forecast 1 day ahead

# Features: all numeric columns except metadata and target
meta_cols = ["Date", "Name", "Symbol", "SourceFile"]
feature_cols = [c for c in df.columns if c not in meta_cols + [TARGET_COL]]

print("\nUsing feature columns:")
print(feature_cols)

# ----------------- Helpers -----------------
def make_sequences(values_X, values_y, lookback, horizon=1):
    """
    Turn time-ordered arrays into (X, y) sequences.
    X[i] is a sequence of 'lookback' time steps.
    y[i] is the target at time t + horizon - 1.
    """
    X_seq, y_seq = [], []
    n = len(values_y)
    for t in range(lookback, n - horizon + 1):
        X_seq.append(values_X[t - lookback:t, :])
        y_seq.append(values_y[t + horizon - 1])
    return np.array(X_seq), np.array(y_seq)


def build_lstm_model(input_shape):
    """
    Simple but solid LSTM model with dropout.
    input_shape = (timesteps, n_features)
    """
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(32, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1))  # predict next Close

    model.compile(
        loss="mse",
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        metrics=["mae"]
    )
    return model


def plot_and_save_forecast(dates, y_true, y_pred, coin, model_name):
    plt.figure(figsize=(11, 5))
    plt.plot(dates, y_true, label="Actual", linewidth=2)
    plt.plot(dates, y_pred, label="Predicted", linewidth=2)
    plt.title(f"{model_name} Forecast vs Actual — {coin}")
    plt.xlabel("Date")
    plt.ylabel("Close Price")
    plt.legend()
    plt.tight_layout()
    out_path = FIG_DIR / f"{model_name}_{coin}_forecast.png"
    plt.savefig(out_path, dpi=240)
    plt.close()
    return out_path


def plot_and_save_residuals(dates, residuals, coin, model_name):
    plt.figure(figsize=(11, 5))
    plt.plot(dates, residuals, color="purple")
    plt.axhline(0, color="black", linestyle="--")
    plt.title(f"{model_name} Residuals — {coin}")
    plt.xlabel("Date")
    plt.ylabel("Residual (y_true - y_pred)")
    plt.tight_layout()
    out_path = FIG_DIR / f"{model_name}_{coin}_residuals.png"
    plt.savefig(out_path, dpi=240)
    plt.close()
    return out_path


def plot_and_save_loss(history, coin, model_name):
    plt.figure(figsize=(8, 4))
    plt.plot(history.history["loss"], label="Train Loss")
    if "val_loss" in history.history:
        plt.plot(history.history["val_loss"], label="Val Loss")
    plt.title(f"{model_name} Loss Curves — {coin}")
    plt.xlabel("Epoch")
    plt.ylabel("MSE")
    plt.legend()
    plt.tight_layout()
    out_path = FIG_DIR / f"{model_name}_{coin}_loss.png"
    plt.savefig(out_path, dpi=240)
    plt.close()
    return out_path


def save_predictions_csv(dates, y_true, y_pred, coin, model_name):
    out = pd.DataFrame({
        "Date": dates,
        "y_true": y_true,
        "y_pred": y_pred
    })
    out_path = PROCESSED_DIR / f"predictions_{model_name.lower()}_{coin}.csv"
    out.to_csv(out_path, index=False)
    return out_path


# ----------------- Main LSTM loop (per coin) -----------------
results_lstm = []

print("\n================= LSTM FORECASTING START =================\n")

for coin in df["Name"].unique():
    print("----------------------------------------------------------")
    print(f" Now Processing Coin: {coin.upper()}")
    print("----------------------------------------------------------")

    sub = df[df["Name"] == coin].copy().dropna(subset=[TARGET_COL])

    if len(sub) < 365:  # at least 1 year of data
        print(f" Skipping {coin}: not enough rows.")
        continue

    sub = sub.sort_values("Date").reset_index(drop=True)

    # Extract features & target
    X_all = sub[feature_cols].values.astype(float)
    y_all = sub[TARGET_COL].values.astype(float)
    dates_all = sub["Date"].values

    n = len(sub)
    n_train = int(n * 0.7)
    n_val = int(n * 0.15)
    n_test = n - n_train - n_val

    train_X_raw = X_all[:n_train]
    val_X_raw   = X_all[n_train:n_train + n_val]
    test_X_raw  = X_all[n_train + n_val:]

    train_y_raw = y_all[:n_train]
    val_y_raw   = y_all[n_train:n_train + n_val]
    test_y_raw  = y_all[n_train + n_val:]

    train_dates = dates_all[:n_train]
    val_dates   = dates_all[n_train:n_train + n_val]
    test_dates  = dates_all[n_train + n_val:]

    print(f"• Train rows: {len(train_X_raw)}, Val: {len(val_X_raw)}, Test: {len(test_X_raw)}")

    # ------------ Scaling (fit only on TRAIN) ------------
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    train_X_scaled = scaler_X.fit_transform(train_X_raw)
    val_X_scaled   = scaler_X.transform(val_X_raw)
    test_X_scaled  = scaler_X.transform(test_X_raw)

    train_y_scaled = scaler_y.fit_transform(train_y_raw.reshape(-1, 1)).flatten()
    val_y_scaled   = scaler_y.transform(val_y_raw.reshape(-1, 1)).flatten()
    test_y_scaled  = scaler_y.transform(test_y_raw.reshape(-1, 1)).flatten()

    # ------------ Sequence building ------------
    X_train_seq, y_train_seq = make_sequences(train_X_scaled, train_y_scaled, LOOKBACK, HORIZON)
    X_val_seq,   y_val_seq   = make_sequences(val_X_scaled,   val_y_scaled,   LOOKBACK, HORIZON)
    X_test_seq,  y_test_seq  = make_sequences(test_X_scaled,  test_y_scaled,  LOOKBACK, HORIZON)

    # Align dates with sequences (use last date in each window + horizon)
    train_seq_dates = train_dates[LOOKBACK - 1:LOOKBACK - 1 + len(y_train_seq)]
    val_seq_dates   = val_dates[LOOKBACK - 1:LOOKBACK - 1 + len(y_val_seq)]
    test_seq_dates  = test_dates[LOOKBACK - 1:LOOKBACK - 1 + len(y_test_seq)]

    print(f"Training samples: {X_train_seq.shape[0]}, "
          f"Validation samples: {X_val_seq.shape[0]}, "
          f"Testing samples: {X_test_seq.shape[0]}")

    if X_train_seq.shape[0] == 0 or X_test_seq.shape[0] == 0:
        print(f" Skipping {coin}: not enough sequence data after LOOKBACK.")
        continue

    # ------------ Build model ------------
    input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
    model = build_lstm_model(input_shape)
    model.summary(print_fn=lambda x: print("   " + x))

    # ------------ Train model ------------
    es = EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

    history = model.fit(
        X_train_seq, y_train_seq,
        validation_data=(X_val_seq, y_val_seq),
        epochs=100,
        batch_size=32,
        callbacks=[es],
        verbose=1
    )

    # ------------ Evaluate ------------
    # Train preds
    y_train_pred_scaled = model.predict(X_train_seq)
    y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled).flatten()
    y_train_true = scaler_y.inverse_transform(y_train_seq.reshape(-1, 1)).flatten()

    train_mae = mean_absolute_error(y_train_true, y_train_pred)
    train_rmse = root_mean_squared_error(y_train_true, y_train_pred)

    # Test preds
    y_test_pred_scaled = model.predict(X_test_seq)
    y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled).flatten()
    y_test_true = scaler_y.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

    test_mae = mean_absolute_error(y_test_true, y_test_pred)
    test_rmse = root_mean_squared_error(y_test_true, y_test_pred)

    print(f"   Train MAE  = {train_mae:.3f}")
    print(f"   Train RMSE = {train_rmse:.3f}")
    print(f"   Test MAE   = {test_mae:.3f}")
    print(f"   Test RMSE  = {test_rmse:.3f}")

    # ------------ Plots & CSVs ------------
    forecast_plot = plot_and_save_forecast(
        dates=test_seq_dates,
        y_true=y_test_true,
        y_pred=y_test_pred,
        coin=coin,
        model_name="LSTM_Multi"
    )

    residuals = y_test_true - y_test_pred
    residual_plot = plot_and_save_residuals(
        dates=test_seq_dates,
        residuals=residuals,
        coin=coin,
        model_name="LSTM_Multi"
    )

    loss_plot = plot_and_save_loss(
        history=history,
        coin=coin,
        model_name="LSTM_Multi"
    )

    pred_csv = save_predictions_csv(
        dates=test_seq_dates,
        y_true=y_test_true,
        y_pred=y_test_pred,
        coin=coin,
        model_name="LSTM_Multi"
    )

    # ------------ Collect results ------------
    results_lstm.append({
        "Coin": coin,
        "Model": "LSTM_Multi",
        "Train_MAE": train_mae,
        "Train_RMSE": train_rmse,
        "Test_MAE": test_mae,
        "Test_RMSE": test_rmse,
        "ForecastPlot": str(forecast_plot),
        "ResidualPlot": str(residual_plot),
        "LossPlot": str(loss_plot),
        "PredCSV": str(pred_csv),
    })

print("\n================= LSTM FORECASTING FINISHED =================\n")

if results_lstm:
    lstm_df = pd.DataFrame(results_lstm).sort_values("Test_RMSE")
    display(lstm_df)
else:
    print("No LSTM results — something went wrong.")


Dataset shape: (37082, 44)
Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Name', 'Symbol', 'SourceFile', 'LogReturn', 'Return_%', 'Volatility_7d', 'Volatility_30d', 'Momentum_7d', 'Momentum_30d', 'RSI_14', 'EMA_12', 'EMA_26', 'MACD', 'MACD_Signal', 'EMA_10', 'EMA_20', 'EMA_50', 'BB_Upper', 'BB_Lower', 'BB_Width', 'High_Low_%', 'Close_Open_%', 'MarketPressure', 'Close_lag1', 'Volume_lag1', 'Return_lag1', 'Close_lag7', 'Volume_lag7', 'Return_lag7', 'Close_lag14', 'Volume_lag14', 'Return_lag14', 'Close_lag30', 'Volume_lag30', 'Return_lag30', 'DayOfWeek', 'Month', 'Quarter']

Using feature columns:
['Open', 'High', 'Low', 'Volume', 'LogReturn', 'Return_%', 'Volatility_7d', 'Volatility_30d', 'Momentum_7d', 'Momentum_30d', 'RSI_14', 'EMA_12', 'EMA_26', 'MACD', 'MACD_Signal', 'EMA_10', 'EMA_20', 'EMA_50', 'BB_Upper', 'BB_Lower', 'BB_Width', 'High_Low_%', 'Close_Open_%', 'MarketPressure', 'Close_lag1', 'Volume_lag1', 'Return_lag1', 'Close_lag7', 'Volume_lag7', 'Return_lag7', 'Clo

   Model: "sequential"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 60, 64)              │          26,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 60, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────

   Model: "sequential_1"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 60, 64)              │          26,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 60, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼───────────────

   Model: "sequential_2"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                        │ (None, 60, 64)              │          26,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 60, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼───────────────

   Model: "sequential_3"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                        │ (None, 60, 64)              │          26,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 60, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼───────────────

...

   Model: "sequential_16"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_32 (LSTM)                       │ (None, 60, 64)              │          26,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_32 (Dropout)                 │ (None, 60, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_33 (LSTM)                       │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_33 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

   Model: "sequential_17"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_34 (LSTM)                       │ (None, 60, 64)              │          26,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_34 (Dropout)                 │ (None, 60, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_35 (LSTM)                       │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_35 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

   Model: "sequential_18"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_36 (LSTM)                       │ (None, 60, 64)              │          26,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_36 (Dropout)                 │ (None, 60, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_37 (LSTM)                       │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_37 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

   Model: "sequential_19"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_38 (LSTM)                       │ (None, 60, 64)              │          26,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_38 (Dropout)                 │ (None, 60, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_39 (LSTM)                       │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_39 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

,Coin,Model,Train_MAE,Train_RMSE,Test_MAE,Test_RMSE,ForecastPlot,ResidualPlot,LossPlot,PredCSV
16,Tether,LSTM_Multi,0.004334,0.009931,0.000603,0.000973,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
17,USD Coin,LSTM_Multi,0.005690,0.008674,0.000458,0.001243,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
15,TRON,LSTM_Multi,0.012693,0.021703,0.057335,0.066134,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
5,Crypto.com Coin,LSTM_Multi,0.032148,0.042713,0.072207,0.080060,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
6,Dogecoin,LSTM_Multi,0.001475,0.001997,0.094590,0.177694,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
12,NEM,LSTM_Multi,0.117143,0.226002,0.191073,0.240598,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
14,Stellar,LSTM_Multi,0.092437,0.128224,0.216501,0.279340,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
19,XRP,LSTM_Multi,0.225008,0.341546,0.346757,0.519098,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
9,IOTA,LSTM_Multi,0.538097,1.026150,1.076793,1.192605,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
2,Cardano,LSTM_Multi,0.096505,0.155733,1.224508,1.253645,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...,C:\Users\BALA\OneDrive - University of Hertfor...
